In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
import json

In [2]:
df = pd.read_csv('sales_adverts.csv', encoding='utf-8')

In [2]:
df2 = pd.read_csv('monthly_rent.csv', encoding='utf-8')

df2

,Unnamed: 0,district,address,room_count,area,floors,house_type,price,latitude,longitude,quadrant,price_psqm
0,2,Čiekurkalns,Rusova 30,1,36.0,3/5,M. ģim.,280.0,56.986648,24.174007,1369.0,7.777778
1,5,Teika,Brīvības 231,1,34.0,4/5,103.,290.0,56.976679,24.176255,1243.0,8.529412
2,17,centrs,Veru 3,2,38.0,1/5,Renov.,450.0,56.961845,24.101000,1039.0,11.842105
3,23,Pļavnieki,Kaudzīšu 6,1,33.0,2/9,602.,220.0,56.938987,24.198513,744.0,6.666667
4,25,Vecmīlgrāvis,Vecmīlgrāvja 1. l. 49,2,50.0,4/5,LT proj.,250.0,57.028859,24.107060,1859.0,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
806,4982,centrs,Rūpniecības 34А,4,146.0,3/6,Jaun.,1200.0,56.965975,24.104628,1103.0,8.219178
807,5004,centrs,Čaka 91,1,10.0,3/4,P. kara,140.0,56.957181,24.137762,984.0,14.000000
808,5011,centrs,Tallinas 33,1,31.0,1/5,P. kara,280.0,56.962705,24.138035,1047.0,9.032258
809,5013,Pļavnieki,Kupriču 1A,2,54.0,3/5,Jaun.,630.0,56.938434,24.224635,749.0,11.666667


In [5]:
with open('districts.json', encoding='utf-8') as f:
    districts = json.load(f)

In [6]:
districts

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'gid': 59, 'apkaime': 'Rumbula'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[24.207479354035634, 56.898296289483774],
      [24.211146704304895, 56.89670453163311],
      [24.213277060647524, 56.89821016967172],
      [24.21659105407348, 56.90022579428325],
      [24.217320481491445, 56.90082194144661],
      [24.21753186119787, 56.901544090690756],
      [24.224709138399255, 56.89800796397556],
      [24.228047476521333, 56.896616479038535],
      [24.227986035858564, 56.89658246217932],
      [24.230516839346212, 56.895283373166556],
      [24.231596503798407, 56.89472942047657],
      [24.23456301565925, 56.893205351063344],
      [24.236325749645466, 56.89229925991029],
      [24.23629146564725, 56.892279273173195],
      [24.238353328036922, 56.8912382426632],
      [24.23872174852832, 56.89105221876954

In [6]:
district_polygons = {}

    
for feature in districts['features']:
    district = feature['properties']['apkaime']
    coordinates = feature['geometry']['coordinates']
    polygon = Polygon(coordinates[0])  # Assuming the coordinates are in the correct format
    district_polygons[district] = polygon


In [8]:
from shapely.geometry import Point

In [38]:
for index, row in df.iterrows():
    point = Point(row['longitude'], row['latitude'])
    for district, polygon in district_polygons.items():
        if polygon.contains(point):
            df.loc[index, 'apkaime'] = district
            break

In [9]:
for index, row in df2.iterrows():
    point = Point(row['longitude'], row['latitude'])
    for district, polygon in district_polygons.items():
        if polygon.contains(point):
            df2.loc[index, 'apkaime'] = district
            break

In [10]:
df2.loc[df2['apkaime'].isna(), 'apkaime']= df2.loc[df2['apkaime'].isna(), 'district']

In [47]:
df.loc[df['apkaime'].isna(), 'apkaime']= df.loc[df['apkaime'].isna(), 'district']

In [48]:
df

,Unnamed: 0,district,address,room_count,area,floors,house_type,price,latitude,longitude,quadrant,price_psqm,apkaime
0,0,Iļģuciems,Palangas 7,3,66.0,5/5,LT proj.,60500.0,56.970871,24.053865,1156.0,916.666667,Iļģuciems
1,1,Purvciems,Varavīksnes g. 14,1,31.0,3/9,602.,40000.0,56.952195,24.204837,934.0,1290.322581,Purvciems
2,2,Pļavnieki,Jasmuižas 6,2,50.0,4/9,602.,51800.0,56.936898,24.209265,746.0,1036.000000,Pļavnieki
3,3,Purvciems,Burtnieku 35,1,13.0,3/4,Hrušč.,18500.0,56.966420,24.181675,1118.0,1423.076923,Teika
4,4,Zolitūde,Ruses 20,2,59.0,5/6,119.,64500.0,56.946474,24.005809,831.0,1093.220339,Zolitūde
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,2574,Vecrīga,Smilšu 14,2,53.0,2/5,P. kara,158800.0,56.950633,24.107169,914.0,2996.226415,Vecpilsēta
2572,2575,Bolderāja,Lielupes 1/2,3,56.0,1/2,Staļina,62000.0,57.032720,24.058551,1913.0,1107.142857,Bolderāja
2573,2576,Ķengarags,Maskavas 297,2,45.0,2/12,Čehu pr.,47900.0,56.914166,24.176720,424.0,1064.444444,Ķengarags
2574,2577,Imanta,Dammes 24,3,70.0,5/5,LT proj.,88000.0,56.964561,24.013022,1022.0,1257.142857,Imanta


In [55]:
df_sell_grouped = df.groupby('apkaime')

In [11]:
df_month_grouped = df2.groupby('apkaime')

In [12]:
monthly_average_vals = df_month_grouped['price_psqm'].mean().sort_values()

In [56]:
average_vals = df_sell_grouped['price_psqm'].mean().sort_values()

In [57]:
average_vals

apkaime
Voleri                   441.122924
Dārziņi                  444.444444
Jaunciems                706.085845
Mangaļsala               796.747743
Daugavgrīva              815.272074
Bolderāja                822.249933
Vecdaugava               843.103448
Vecmilgrāvis             881.519456
Sarkandaugava            958.207402
Dzirciems                965.202598
Iļģuciems                968.419181
Šķirotava               1012.755799
Rumbula                 1080.542333
Ķengarags               1090.560417
Dārzciems               1116.758625
Šampēteris              1166.372167
Jugla                   1167.261294
Bieriņi                 1192.559807
Zolitūde                1207.176808
Pļavnieki               1208.799380
Katlakalns              1221.038191
Ziepniekkalns           1243.465557
Purvciems               1246.462473
Čiekurkalns             1249.284994
Bišumuiža               1250.000000
Imanta                  1305.849934
Torņakalns              1374.830301
Bukulti             

In [58]:
df.to_csv('district_avg_psqm.csv')

In [15]:
df2.to_csv('monthly_avg_psqm.csv')